In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
import optuna
import torch

from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataModule, CHBMITDataset

torch.set_float32_matmul_precision("high")

2025-05-18 15:57:20.790 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /home/jupyter-group55/snn-encoder-test


In [3]:
# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

In [4]:
datamodule = CHBMITDataModule(dataset, batch_size=512, worker=0)

In [5]:
tuning_trials = 200

In [6]:
tuning_list = ["be"]

In [7]:
import gc
import os

from IPython.display import clear_output
from loguru import logger

from eeg_snn_encoder.tuning import create_objective

for tuning in tuning_list:
    clear_output(wait=True)
    logger.info(f"Starting tuning for {tuning}")

    encoder_study = optuna.load_study(
        study_name=f"encoder-tuning-{tuning}",
        storage=os.environ["OPTUNA_CONN_STRING_CPE"],
    )

    fixed_params = encoder_study.best_params

    del encoder_study

    sampler = optuna.samplers.TPESampler()
    fixed_sampler = optuna.samplers.PartialFixedSampler(
        fixed_params=fixed_params, base_sampler=sampler
    )

    pruner = optuna.pruners.HyperbandPruner()

    study = optuna.create_study(
        direction="minimize",
        study_name=f"model-tuning-{tuning}-model-only",
        storage=os.environ["OPTUNA_CONN_STRING_CPE"],
        load_if_exists=True,
        sampler=fixed_sampler,
        pruner=pruner,
    )

    objective = create_objective(
        encoder_type=tuning,
        datamodule=datamodule,
        moniter_metric="val_mse",
        moniter_mode="min",
    )

    try:
        study.optimize(objective, n_trials=tuning_trials, gc_after_trial=True)
        logger.info(f"Best value for {tuning}: {study.best_value}")
        logger.info(f"Best params for {tuning}: {study.best_params}")

        logger.info(f"Complete tuning {tuning} for {tuning_trials} trials")
    except Exception as e:
        logger.exception(f"Error occurred during optimization for {tuning}: {e}")

    del study
    torch.cuda.empty_cache()
    gc.collect()

2025-05-18 15:57:22.359 | INFO     | __main__:<module>:11 - Starting tuning for be


/tmp/ipykernel_938399/2379794623.py:23: ExperimentalWarning: PartialFixedSampler is experimental (supported from v2.4.0). The interface can change in the future.
  fixed_sampler = optuna.samplers.PartialFixedSampler(
[I 2025-05-18 15:57:22,831] Using an existing study with name 'model-tuning-be-model-only' instead of creating a new one.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


New best val_mse: 0.5192562937736511 | Old best: inf


...

New best val_mse: 0.42496681213378906 | Old best: 0.45683932304382324


Validation: |          | 0/? [00:00<?, ?it/s]

New best val_mse: 0.41035857796669006 | Old best: 0.42496681213378906


[I 2025-05-18 16:13:05,963] Trial 301 pruned. Trial was pruned at epoch 3.


2025-05-18 16:13:06.136 | INFO     | __main__:<module>:47 - Best value for be: 0.1447543203830719
2025-05-18 16:13:06.157 | INFO     | __main__:<module>:48 - Best params for be: {'threshold': 0.04672637549075715, 'slope': 13.91883541276826, 'beta': 0.7852920828254439, 'dropout_rate1': 0.16186461262366283, 'dropout_rate2': 0.4596067226217043, 'lr': 9.866050942144134e-05, 'weight_decay': 1.5134463291603487e-05, 'scheduler_factor': 0.6140026252145396, 'scheduler_patience': 3, 'burst_max_window': 8, 'burst_n_max': 7, 'burst_t_max': 0, 'burst_t_min': 0}
2025-05-18 16:13:06.157 | INFO     | __main__:<module>:50 - Complete tuning be for 36 trials
